In [1]:
# !pip install datasets
# !pip install urllib3
# !pip uninstall urllib3
# !pip install --upgrade botocore
# !pip install --upgrade datasets
# !pip install -U datasets
# !pip install python-terrier

In [2]:
!python --version

Python 3.11.5


In [3]:
# !python -m venv rag_eval

# IMPORTS

In [4]:
from datasets import load_dataset
from tqdm.auto import tqdm
from utils.metrics_utils import *
import numpy as np

In [5]:
tqdm.pandas()

# Dowload dataset

In [6]:
# Загрузка конфигурации question-answer-passages
qa_dataset = load_dataset("enelpol/rag-mini-bioasq", "question-answer-passages")

# Загрузка конфигурации text-corpus
text_dataset = load_dataset("enelpol/rag-mini-bioasq", "text-corpus")

In [7]:
df_rag = qa_dataset["train"].to_pandas()

In [8]:
indexed_passages = text_dataset["test"].to_pandas()

In [9]:
display(df_rag.head(2), indexed_passages.head(2))

,question,answer,id,relevant_passage_ids
0,What is the implication of histone lysine meth...,"Aberrant patterns of H3K4, H3K9, and H3K27 his...",1682,"[23179372, 19270706, 23184418]"
1,What is the role of STAG1/STAG2 proteins in di...,STAG1/STAG2 proteins are tumour suppressor pro...,3722,"[26997282, 21589869, 19822671, 29867216, 15361..."


,passage,id
0,New data on viruses isolated from patients wit...,9797
1,We describe an improved method for detecting d...,11906


In [10]:
def generate_from_retriever(row, indexed_passages, prob=0.85, top_k=20):
    """
    Создает столбец 'from_retriever', имитируя поиск и ранжирование.
    
    Параметры:
    row - строка из DataFrame с колонками 'question', 'answer', 'id', 'relevant_passage_ids'.
    indexed_passages - DataFrame с проиндексированными пассажами.
    prob - вероятность добавления релевантного пассажа.
    top_k - количество возвращаемых пассажей (топ-K).
    
    Возвращает:
    Список из top_k документов (ID), который содержит релевантные документы с вероятностью 'prob' и остальные документы.
    """
    relevant_passage_ids = row['relevant_passage_ids']
    from_retriever = []

    for passage_id in relevant_passage_ids:
        if np.random.rand() <= prob:
            from_retriever.append(passage_id)

    if len(from_retriever) < top_k:
        available_ids = set(indexed_passages['id']) - set(from_retriever)
        additional_passages = list(np.random.choice(list(available_ids), size=(top_k - len(from_retriever)), replace=False))
        from_retriever.extend(additional_passages)
    
    np.random.shuffle(from_retriever)

    return from_retriever

In [11]:
df_rag['from_retriever'] = df_rag.progress_apply(generate_from_retriever, args=(indexed_passages,), axis=1)

  0%|          | 0/4012 [00:00<?, ?it/s]

In [12]:
df_rag.head(2)

,question,answer,id,relevant_passage_ids,from_retriever
0,What is the implication of histone lysine meth...,"Aberrant patterns of H3K4, H3K9, and H3K27 his...",1682,"[23179372, 19270706, 23184418]","[26501573, 27279928, 23429751, 23179372, 18926..."
1,What is the role of STAG1/STAG2 proteins in di...,STAG1/STAG2 proteins are tumour suppressor pro...,3722,"[26997282, 21589869, 19822671, 29867216, 15361...","[1445915, 11826022, 18511909, 26243569, 153618..."


In [13]:
def generate_from_ranker(row, prob=0.8):
    """
    Создает столбец 'from_ranker', имитируя ранжирование результатов поиска.
    
    Параметры:
    row - строка из DataFrame с колонками 'from_retriever' и 'relevant_passage_ids'.
    prob - базовая вероятность поднятия релевантного документа в топ списка.
    
    Возвращает:
    Список документов (ID) с ранжированием, где релевантные документы поднимаются выше с увеличенной вероятностью.
    """
    from_retriever = row['from_retriever']
    relevant_passage_ids = set(row['relevant_passage_ids'])
    from_ranker = from_retriever.copy()

    for i in range(len(from_ranker) - 1, -1, -1):
        if from_ranker[i] in relevant_passage_ids:
            if np.random.rand() <= prob:
                weights = np.array([1 / (j + 1) for j in range(i + 1)])  # Инвертированное линейное распределение
                weights = weights / weights.sum()  # Нормализация весов для получения вероятностей
                new_position = np.random.choice(range(i + 1), p=weights)
                from_ranker.insert(new_position, from_ranker.pop(i))
    
    return from_ranker

In [14]:
df_rag['from_ranker'] = df_rag.progress_apply(generate_from_ranker, axis=1)

  0%|          | 0/4012 [00:00<?, ?it/s]

In [15]:
df_rag.head(2)

,question,answer,id,relevant_passage_ids,from_retriever,from_ranker
0,What is the implication of histone lysine meth...,"Aberrant patterns of H3K4, H3K9, and H3K27 his...",1682,"[23179372, 19270706, 23184418]","[26501573, 27279928, 23429751, 23179372, 18926...","[23179372, 23184418, 26501573, 27279928, 23429..."
1,What is the role of STAG1/STAG2 proteins in di...,STAG1/STAG2 proteins are tumour suppressor pro...,3722,"[26997282, 21589869, 19822671, 29867216, 15361...","[1445915, 11826022, 18511909, 26243569, 153618...","[21589869, 26997282, 19822671, 12034751, 14459..."


=============================================================================================================

# МЕТРИКИ БЕЗ УЧЕТА РАНЖИРОВАНИЯ

## PRECISION@K

In [16]:
df_rag = calculate_precision(df_rag, "from_retriever", "relevant_passage_ids", output_col='Precision', k=None)
df_rag.head(2)

  0%|          | 0/4012 [00:00<?, ?it/s]

,question,answer,id,relevant_passage_ids,from_retriever,from_ranker,Precision
0,What is the implication of histone lysine meth...,"Aberrant patterns of H3K4, H3K9, and H3K27 his...",1682,"[23179372, 19270706, 23184418]","[26501573, 27279928, 23429751, 23179372, 18926...","[23179372, 23184418, 26501573, 27279928, 23429...",0.10
1,What is the role of STAG1/STAG2 proteins in di...,STAG1/STAG2 proteins are tumour suppressor pro...,3722,"[26997282, 21589869, 19822671, 29867216, 15361...","[1445915, 11826022, 18511909, 26243569, 153618...","[21589869, 26997282, 19822671, 12034751, 14459...",0.35


In [17]:
df_rag = precision_sklearn(df_rag, "from_retriever", "relevant_passage_ids", output_col='Precision_sk', k=None)
df_rag.head(2)

  0%|          | 0/4012 [00:00<?, ?it/s]

,question,answer,id,relevant_passage_ids,from_retriever,from_ranker,Precision,Precision_sk
0,What is the implication of histone lysine meth...,"Aberrant patterns of H3K4, H3K9, and H3K27 his...",1682,"[23179372, 19270706, 23184418]","[26501573, 27279928, 23429751, 23179372, 18926...","[23179372, 23184418, 26501573, 27279928, 23429...",0.10,0.10
1,What is the role of STAG1/STAG2 proteins in di...,STAG1/STAG2 proteins are tumour suppressor pro...,3722,"[26997282, 21589869, 19822671, 29867216, 15361...","[1445915, 11826022, 18511909, 26243569, 153618...","[21589869, 26997282, 19822671, 12034751, 14459...",0.35,0.35


In [18]:
for k in [20, 10, 5, 3]:
    df_rag = precision_sklearn(df_rag, "from_retriever", "relevant_passage_ids", output_col=f'Precision_sk_{k}', k=k)
    print(f"mean_precision_sklearn_{k} = {round(df_rag[f'Precision_sk_{k}'].mean(), 3)}")
df_rag.head(2)

  0%|          | 0/4012 [00:00<?, ?it/s]

mean_precision_sklearn_20 = 0.355


  0%|          | 0/4012 [00:00<?, ?it/s]

mean_precision_sklearn_10 = 0.356


  0%|          | 0/4012 [00:00<?, ?it/s]

mean_precision_sklearn_5 = 0.355


  0%|          | 0/4012 [00:00<?, ?it/s]

mean_precision_sklearn_3 = 0.36


,question,answer,id,relevant_passage_ids,from_retriever,from_ranker,Precision,Precision_sk,Precision_sk_20,Precision_sk_10,Precision_sk_5,Precision_sk_3
0,What is the implication of histone lysine meth...,"Aberrant patterns of H3K4, H3K9, and H3K27 his...",1682,"[23179372, 19270706, 23184418]","[26501573, 27279928, 23429751, 23179372, 18926...","[23179372, 23184418, 26501573, 27279928, 23429...",0.10,0.10,0.10,0.1,0.2,0.0
1,What is the role of STAG1/STAG2 proteins in di...,STAG1/STAG2 proteins are tumour suppressor pro...,3722,"[26997282, 21589869, 19822671, 29867216, 15361...","[1445915, 11826022, 18511909, 26243569, 153618...","[21589869, 26997282, 19822671, 12034751, 14459...",0.35,0.35,0.35,0.5,0.2,0.0


In [19]:
for k in [20, 10, 5, 3]:
    df_rag = calculate_precision(df_rag, "from_retriever", "relevant_passage_ids", output_col=f'Precision_{k}', k=k)
    print(f"mean_precision_{k} = {round(df_rag[f'Precision_{k}'].mean(), 3)}")
df_rag.head(2)

  0%|          | 0/4012 [00:00<?, ?it/s]

mean_precision_20 = 0.355


  0%|          | 0/4012 [00:00<?, ?it/s]

mean_precision_10 = 0.356


  0%|          | 0/4012 [00:00<?, ?it/s]

mean_precision_5 = 0.355


  0%|          | 0/4012 [00:00<?, ?it/s]

mean_precision_3 = 0.36


,question,answer,id,relevant_passage_ids,from_retriever,from_ranker,Precision,Precision_sk,Precision_sk_20,Precision_sk_10,Precision_sk_5,Precision_sk_3,Precision_20,Precision_10,Precision_5,Precision_3
0,What is the implication of histone lysine meth...,"Aberrant patterns of H3K4, H3K9, and H3K27 his...",1682,"[23179372, 19270706, 23184418]","[26501573, 27279928, 23429751, 23179372, 18926...","[23179372, 23184418, 26501573, 27279928, 23429...",0.10,0.10,0.10,0.1,0.2,0.0,0.10,0.1,0.2,0.0
1,What is the role of STAG1/STAG2 proteins in di...,STAG1/STAG2 proteins are tumour suppressor pro...,3722,"[26997282, 21589869, 19822671, 29867216, 15361...","[1445915, 11826022, 18511909, 26243569, 153618...","[21589869, 26997282, 19822671, 12034751, 14459...",0.35,0.35,0.35,0.5,0.2,0.0,0.35,0.5,0.2,0.0


In [ ]:
for k in [20, 10, 5, 3]:
    df_rag = precision_sklearn(df_rag, "from_ranker", "relevant_passage_ids", output_col=f'Precision_rank{k}', k=k)
    print(f"precision_sk_rank{k} = {round(df_rag[f'Precision_rank{k}'].mean(), 3)}")
df_rag.head(2)

  0%|          | 0/4012 [00:00<?, ?it/s]

precision_sk_rank20 = 0.355


  0%|          | 0/4012 [00:00<?, ?it/s]

precision_sk_rank10 = 0.457


  0%|          | 0/4012 [00:00<?, ?it/s]

precision_sk_rank5 = 0.596


  0%|          | 0/4012 [00:00<?, ?it/s]

In [ ]:
for k in [20, 10, 5, 3]:
    df_rag = calculate_precision(df_rag, "from_ranker", "relevant_passage_ids", output_col=f'Precision_rank_{k}', k=k)
    print(f"precision_rank_{k} = {round(df_rag[f'Precision_rank_{k}'].mean(), 3)}")
df_rag.head(2)

## RECALL@K

In [ ]:
for k in [20, 10, 5, 3]:
    df_rag = recall_sklearn(df_rag, "from_ranker", "relevant_passage_ids", output_col=f'recall_rank{k}', k=k)
    print(f"recall_rank{k} = {round(df_rag[f'recall_rank{k}'].mean(), 3)}")

In [ ]:
for k in [20, 10, 5, 3]:
    df_rag = calculate_recall(df_rag, "from_ranker", "relevant_passage_ids", output_col=f'recall_rank_{k}', k=k)
    print(f"recall_rank{k} = {round(df_rag[f'recall_rank_{k}'].mean(), 3)}")

## F1@K

In [ ]:
for k in [20, 10, 5, 3]:
    df_rag = fbeta_sklearn(df_rag, "from_retriever", "relevant_passage_ids", output_col=f'f1@{k}', k=k)
    print(f"f1@{k} = {round(df_rag[f'f1@{k}'].mean(), 3)}")

=======================================================================================================================

# МЕТРИКИ, УЧИТЫВАЮЩИЕ РАНЖИРОВАНИЕ

## MRR

In [ ]:
df_rag, mrr = calculate_mrr(df_rag, "from_retriever", "relevant_passage_ids", output_col="RR_ret")

In [ ]:
df_rag, mrr = calculate_mrr(df_rag, "from_ranker", "relevant_passage_ids", output_col="RR_ran")

## AP/MAP

In [ ]:
for k in [20, 10, 5, 3]:
    df_rag, _map_1 = calculate_map(df_rag, "from_retriever", "relevant_passage_ids", output_col=f'AP@{k}', k=k)

In [ ]:
for k in [20, 10, 5, 3]:
    df_rag, _map_2 = calculate_map(df_rag, "from_ranker", "relevant_passage_ids", output_col=f'_AP@{k}', k=k)

In [ ]:
for k in [20, 10, 5, 3]:
    df_rag, _map_2 = map_sklearn(df_rag, "from_ranker", "relevant_passage_ids", output_col=f'_AP@{k}', k=k)

## NDCG

In [ ]:
# base

In [ ]:

for k in [20, 10, 5, 3]:
    df_rag, ndcg_1 = calculate_ndcg_at_k(df_rag, "from_retriever", "relevant_passage_ids", output_col=f'NDCG@{k}', k=k)

In [ ]:
for k in [10, 5, 3]:
    df_rag, _map = calculate_ndcg_at_k(df_rag, "from_ranker", "relevant_passage_ids", output_col=f'_NDCG@{k}', k=k)

In [ ]:
# sklearn метрика условно не применима - она учитывает степень релевантности документов, которых в RAG обычно нет, поэтому расчеты отличаются
# от функции выше (она для бинарной релевантности)